In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import re
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import resample
from xgboost import XGBRegressor
from difflib import SequenceMatcher
from rfpimp import permutation_importances

In [2]:
search= input('keyword:')

keyword:funda Iphone 10


In [3]:
def features(key):
    search=key.replace(' ', '+')
    pages=range(20)
    ASIN=[]
    key=[]
    value=[]
    features=[]
    for i in pages:
        url='https://www.amazon.es/s?k='+search+'&page='+str(i+1)+'&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1632992874&ref=sr_pg_'+str(i+1)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "lxml")
        info = soup.find_all('div', {'class': 'sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col sg-col-4-of-20'})
        for link in info:
            ASIN.append(link.get('data-asin'))
    
    for a in ASIN:
        params = {'api_key': 'DC5C88DD62004EFCA8E0CCEF98D6F99C',
                'type': 'product',
                'amazon_domain': 'amazon.es',
                'asin': a}
        api_result = requests.get('https://api.rainforestapi.com/request', params)
        data=json.loads(json.dumps(api_result.json()))
        
        if 'product' in data.keys():
            y= data['product']
            if 'specifications' in y.keys():
                x=y['specifications']
                
                for i in range(len(x)):
                    if x[i]['value']!=None:
                        key.append(x[i]['name'])
                        value.append(x[i]['value'])
                
                if 'rating' in y.keys():
                    if 'buybox_winner' in y.keys():
                        y1=y['buybox_winner']
                        if 'price' in y1.keys():
                            y2=y1['price']
                            if 'value' in y2.keys():
                                key.append('Rating')
                                key.append('Price €')
                                value.append(y['rating'])
                                value.append(y2['value'])
                    
        features.append(dict(zip(key,value)))  
    
    return pd.DataFrame(features)

In [3]:
def clean(DataFrame):
    
    string='\u200e'
    drop_list=[]

    for column in DataFrame.columns:
        if DataFrame[column].isna().sum()/len(DataFrame) > 0.05:
            drop_list.append(column)   
    
    for column in DataFrame.columns:
        if (len(DataFrame[column].value_counts())==1)&(column not in drop_list):
            drop_list.append(column)

    drop_list_id=['ASIN', 'Valoración media de los clientes', 'Clasificación en los más vendidos de Amazon','Referencia del fabricante','Nmero de modelo del producto', 'Clasificacin en los ms vendidos de Amazon', 'Producto en Amazon.es desde', 'Dimensiones del paquete']
    drop_list= drop_list+drop_list_id
    drop_list.append('Dimensiones del producto')
    drop_list_repeated=['Peso del producto','Dimensiones del producto: largo x ancho x alto']
    drop_list= drop_list+drop_list_repeated
    
    size=[]
    weight=[]
    
    for value in DataFrame['Dimensiones del producto']:
        if ';' in str(value):
            size.append(str(value).split(';')[0])
            weight.append(str(value).split(';')[1])
        else:
            size.append(str(value))
            weight.append(np.nan)

    
    largo=[]
    largo_clean=[]
    alto_clean=[]
    weight_clean=[]
    ancho=[]
    alto=[]
    size_sep=[]
    
    
    
    for i in size:
        if i=='nan':
            largo.append(i)
            ancho.append(i)
            alto.append(i)
        else:
            largo.append(i.split('x')[0])
            ancho.append(i.split('x')[1])
            alto.append(i.split('x')[2])
    
    y=[]
    x=[]
    for i in weight:
        if pd.isna(i):
            weight_clean.append(i)
        else:
            weight_clean.append(str(i).split(' ')[1])
            y.append(i.split(' ')[2])
    y=y[0]        
    
    
    for i in alto:
        if i== 'nan':
            alto_clean.append(i)
        else:
            alto_clean.append(str(i).split(' ')[1])
            x.append(i.split(' ')[2])
    x=x[0]
    
    for i in largo:
        if string in i:
            value = i[len(string):]
            largo_clean.append(value)
        else:
            largo_clean.append(i)
    
   
    for i in range(0, len(largo_clean)):
        largo_clean[i] = float(largo_clean[i])
    for i in range(0, len(ancho)):
        ancho[i] = float(ancho[i])
    for i in range(0, len(alto_clean)):
        alto_clean[i] = float(alto_clean[i])
    for i in range(0, len(weight_clean)):
        weight_clean[i] = float(weight_clean[i])
    
    def nan_elim(lists):
        for value in lists:
            if value is np.nan:
                return(np.mean(lists))
            else:
                return value
    
    largo_clean=nan_elim(largo_clean)
    ancho=nan_elim(ancho)
    alto_clean=nan_elim(alto_clean)
    weight_clean=nan_elim(weight_clean)
    
    DataFrame['length '+str(x)]=largo_clean
    DataFrame['width '+str(x)]=ancho
    DataFrame['height '+str(x)]=alto_clean
    DataFrame['weight '+str(y)]=weight_clean

    def similar(DataFrame):
        new_drop=[]
        for column in DataFrame.columns:
            if column != 'Tamaño':
                if SequenceMatcher(None, DataFrame['Tamaño'][0], str(DataFrame[column][0])).ratio() > 0.5:
                    new_drop.append(column)
            
        return new_drop   
    
    if 'Tamaño' in DataFrame.columns:
        drop_list= drop_list+ similar(DataFrame)
    

    
    for column in drop_list:
        if (column in drop_list) & (column in DataFrame.columns):
            DataFrame=DataFrame.drop(column,axis=1)

    DataFrame=DataFrame.reset_index(drop=True)
    dic_NAN=dict(zip(DataFrame.columns,DataFrame.isna().sum()/len(DataFrame)))
    list_NAN=sorted(dic_NAN, key=dic_NAN.get, reverse=True)
    DataFrame=DataFrame[DataFrame[list_NAN[0]].isna()==False]    
    DataFrame=DataFrame.reset_index(drop=True) 
    
    return DataFrame



In [22]:
def output(df):
    
    X=df.drop(['Rating'],axis=1)
    y=df['Rating']
    

    X_cat=X.select_dtypes(include='object')
    X_num=X.select_dtypes(include='number')
    X_cat_encoded= pd.get_dummies(X_cat,drop_first=True)
    X=pd.concat([X_cat_encoded,X_num],axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2,train_size=0.8, random_state = 100)
    
    StandardTransformer = StandardScaler().fit(X_train)
    X_stand = pd.DataFrame(StandardTransformer.transform(X_train), columns = X.columns)
    X_stand_test = pd.DataFrame(StandardTransformer.transform(X_test), columns = X.columns)
    
    clf=RandomForestRegressor(n_estimators=20, criterion='mse', max_depth=15 , min_samples_split=2 )
    clf.fit(X_stand,y_train)
    predictions=clf.predict(X_stand_test)
    r21=clf.score(X_stand,y_train)
    r22=clf.score(X_stand_test,y_test)
    
    def r2(rf, X_train, y_train):
        return r2_score(y_train, rf.predict(X_train))

    perm = permutation_importances(clf, X_train, y_train, r2).reset_index()
    
    feature=[]
    value=[]
    for i in perm['Feature']:
        if ('_') in str(i):
            feature.append(i.split('_')[0])
            value.append(i.split('_')[1])
        else:
            feature.append(i)
            value.append(np.nan)
    
    perm=perm.drop(['Feature'],axis=1)
    perm['feature']=feature
    perm=perm.groupby(['feature']).agg({'Importance':sum}).sort_values(by='Importance',ascending=False).reset_index()
    clean=df.sort_values(by='Rating',ascending=False)
    
    imp_feature=[]
    top_values=[]
    output=[]

    for i in range(5):
        imp_feature.append(perm['feature'][i])
        for n in range(5):
            top_values.append(clean[perm['feature'][i]][n])
    list_of_lists = [top_values[i:i+n] for i in range(0, len(top_values), 5)]

    output.append(dict(zip(imp_feature,list_of_lists)))
    return output

In [6]:
a=features(search)

In [15]:
b=clean(a)

In [23]:
output(b)

[{'Modelos de teléfono compatibles': ['Apple iPhone XS',
   'IPhone XR, IPhone 11',
   'IPhone XS, IPhone X',
   'Apple iPhone X, Apple iPhone XS'],
  'Fabricante': ['\u200eUNBREAKcable',
   'UNBREAKcable',
   '\u200eBERMIEU',
   '\u200eGeestyle'],
  'Color': ['Negro',
   'Negro',
   'Transparente Funda para iPhone X/iPhone XS',
   'Naranja / Verde'],
  'Marca': ['UNBREAKcable', 'UNBREAKcable', 'BERMIEU', 'MobNano'],
  'Función especial': ['Diseño Amigable con el Usuario, Premium Bayer TPU de Alemania, Recortes Precisos, Acabado ultraplano y ligero, Diseño sencillo pero elegante, Diseño antideslizante, Diseño de superficie mate, no hay miedo a amarillear, Protección de Pantalla y CámaraVer más',
   '[Sin Burbujas y sin Huellas], [Contra rayaduras y roturas], [Ajuste perfecto], [Instalación sencillo], [Táctil 3D Touch], [99.9% transparencia]Ver más',
   '[Sin Burbujas y sin Huellas], [Contra rayaduras y roturas], [Ajuste perfecto], [Instalación sencillo], [Táctil 3D Touch], [99.9% trans

In [14]:
df=pd.read_csv('casco.csv').drop(['Unnamed: 0'],axis=1)

In [15]:
df

,Dimensiones del paquete,Producto en Amazon.es desde,Fabricante,ASIN,Nmero de modelo del producto,Pas de origen,Departamento,Clasificacin en los ms vendidos de Amazon,Tamaño,Color,Usos recomendados para el producto,Marca,Tipo de servicio del vehículo,Material del interior,Material para exteriores,Rating,Price,Is Discontinued By Manufacturer,Dimensiones del producto,Peso,Dimensiones del producto: largo x ancho x alto,Rango de edad (descripción),Pilas,Referencia del fabricante,Número de producto,Pilas / baterías incluidas,Pilas / baterías necesarias,Composición de la pila de la batería,Tipo de conector,Peso del producto,Valoración media de los clientes,Clasificación en los más vendidos de Amazon,Nivel de resistencia al agua,Tecnología de conectividad,Modelo,Factor de forma,Función especial,Incluye cubierta,Memoria extraíble,Aparatos compatibles,Tamaño de la pantalla,Tiene estabilizador de imagen,Resolución máxima,Resolución de captura de vídeo,Potencia eléctrica,Restricciones de envío,Material,Tipo de ajuste,Vataje
0,36.9 x 28.2 x 26.1 cm; 2.22 kilogramos,23 diciembre 2020,"Lanxi Yema Motorcycle Fittings Co.,LTD",B07P5Q3GGZ,YEMA-829WXL,China,Niños,Ver el Top 100 en Coche y moto,X-Large,Blanco,Motociclismo,YEMA Helmet,"Scooter, Moto de nieve, Ciclomotor","Poliestireno expandido, ABS",Acrilonitrilo butadieno estireno,4.5,64.99,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36.9 x 28.2 x 26.1 cm; 2.22 kilogramos,19 abril 2017,"Lanxi Yema Motorcycle Fittings Co.,LTD",B071CL27W9,YM-831MBXL,China,Hombre,Ver el Top 100 en Coche y moto,X-Large,Negro Mate,"Snowboarding, Motociclismo, Ciclismo",YEMA Helmet,"Scooter, Moto de nieve, Ciclomotor",Poliestireno expandido,Plástico ABS.,4.5,79.99,No,36 x 27 x 27 cm; 1.6 kilogramos,1.6 Kilogramos,36 x 27 x 27 centímetros,Adulto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36.9 x 28.2 x 26.1 cm; 2.22 kilogramos,19 mayo 2017,"Lanxi Yema Motorcycle Fittings Co.,LTD",B01N3K6V25,YEMA-926MBS,China,Hombres,Ver el Top 100 en Coche y moto,Small,Negro Mate,"Motociclismo, Ciclismo",YEMA Helmet,Moto de nieve,Poliestireno expandido,Acrilonitrilo butadieno estireno,4.4,99.99,No,36.07 x 26.92 x 26.92 cm; 1.7 kilogramos,1.7 Kilogramos,36.1 x 26.9 x 26.9 centímetros,Adulto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36.3 x 27.5 x 25.5 cm; 2.29 kilogramos,9 diciembre 2020,"Lanxi Yema Motorcycle Fittings Co.,LTD",B08FWXQQJD,YEMA-925WXL-2,China,Niños,Ver el Top 100 en Coche y moto,X-Large,Blanco,Motociclismo,YEMA Helmet,"Scooter, Moto de nieve, Ciclomotor","ABS, Poliestireno expandido",Acrilonitrilo butadieno estireno,3.3,79.99,No,36.07 x 26.92 x 26.92 cm; 1.7 kilogramos,1.7 Kilogramos,36.1 x 26.9 x 26.9 centímetros,Adulto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,‎15.39 x 11.2 x 6.4 cm; 372 gramos,29 julio 2020,‎QSPORTPEAK,B08DTY2T8V,YEMA-925WXL-2,China,Niños,Ver el Top 100 en Coche y moto,X-Large,Blanco,Motociclismo,QSPORTPEAK,"Scooter, Moto de nieve, Ciclomotor","ABS, Poliestireno expandido",Acrilonitrilo butadieno estireno,3.5,71.99,No,36.07 x 26.92 x 26.92 cm; 1.7 kilogramos,1.7 Kilogramos,36.1 x 26.9 x 26.9 centímetros,Adulto,‎1 Litio Ion necesaria,‎M2,‎M2,‎Sí,‎Sí,‎Litio Ion,Inalámbrico,‎372 g,"3,5 de 5 estrellas64 valoraciones3,5 de 5 estr...",nº225 en Manos libres Bluetooth para moto,Impermeable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,36.5 x 27.7 x 26 cm; 2.22 kilogramos,4 abril 2019,INTERPHONE,B07QB3Z3Q9,INTERPHOACTIVETP,Filipinas,Hombres,Manos libres Bluetooth para moto,Small,Negro,"Motociclismo, Ciclismo",I

In [19]:
b1=clean(df)

In [20]:
b1

,Fabricante,Pas de origen,Departamento,Tamaño,Color,Usos recomendados para el producto,Marca,Tipo de servicio del vehículo,Material del interior,Material para exteriores,Rating,Price,Peso,Pilas,Número de producto,Pilas / baterías incluidas,Pilas / baterías necesarias
0,‎QSPORTPEAK,China,Niños,X-Large,Blanco,Motociclismo,QSPORTPEAK,"Scooter, Moto de nieve, Ciclomotor","ABS, Poliestireno expandido",Acrilonitrilo butadieno estireno,3.5,71.99,1.7 Kilogramos,‎1 Litio Ion necesaria,‎M2,‎Sí,‎Sí
1,BOSEMAN,China,Niños,X-Large,Rosa,"Motociclismo, Patinete",BOSEMAN,"Motocicleta, Scooter, Bicicleta, Ciclomotor",Poliestireno expandido,Acrilonitrilo butadieno estireno,3.6,51.99,1.7 Kilogramos,‎1 Litio Ion necesaria,‎M2,‎Sí,‎Sí
2,GoolRC,China,Niños,X-Large,Rojo,"Motociclismo, Patinete",GoolRC,Motocicleta,Algodón,Acrilonitrilo butadieno estireno,4.1,60.99,1.7 Kilogramos,‎1 Litio Ion necesaria,‎M2,‎Sí,‎Sí
3,INTERPHONE,Filipinas,Niños,X-Large,Negro,"Motociclismo, Patinete",Interphone,Motocicleta,Algodón,Acrilonitrilo butadieno estireno,1.0,278.39,150 Gramos,"1 Polímero de litio necesaria(s), incluida(s)",‎M2,‎Sí,‎Sí
4,Arcweg,China,Hombre,X-Large,Negro,"Motociclismo, Patinete",Interphone,Motocicleta,Algodón,Acrilonitrilo butadieno estireno,4.2,8.99,150 Gramos,"1 Polímero de litio necesaria(s), incluida(s)",‎M2,‎Sí,‎Sí
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,INTERPHONE,Filipinas,Hombres,Small,Negro,"Motociclismo, Ciclismo",Interphone,Moto de nieve,Poliestireno expandido,Acrilonitrilo butadieno estireno,1.0,278.39,150 Gramos,"1 Polímero de litio necesaria(s), incluida(s)",‎M2,‎Sí,‎Sí
232,Arcweg,China,Hombre,Small,Negro,"Motociclismo, Ciclismo",Interphone,Moto de nieve,Poliestireno expandido,Acrilonitrilo butadieno estireno,4.2,8.99,150 Gramos,"1 Polímero de litio necesaria(s), incluida(s)",‎M2,‎Sí,‎Sí
233,XUEER,China,Hombre,Large,B,"Motociclismo, Ciclismo",XUEER,"Moto de cross, ATV, Motocicleta, Bicicleta",Poliestireno expandido,Acrilonitrilo butadieno estireno,4.2,8.99,150 Gramos,"1 Polímero de litio necesaria(s), incluida(s)",‎M2,‎Sí,‎Sí
234,LIONCIANO,China,Hombre,L=57-58cm,Negro Mate Ruta 66,Motociclismo,LIONCIANO,"Motocicleta, Scooter, Ciclomotor",Poliestireno expandido,Acrilonitrilo butadieno estireno,4.2,8.99,150 Gramos,"1 Polímero de litio necesaria(s), incluida(s)",‎M2,‎Sí,‎Sí


In [25]:
b1.to_csv('casco2.csv')

In [23]:
o=output(b1)

In [24]:
o

[{'Color': ['Blanco', 'Rosa', 'Rojo', 'Negro'],
  'Departamento': ['Niños', 'Niños', 'Niños', 'Niños'],
  'Fabricante': ['\u200eQSPORTPEAK', 'BOSEMAN', 'GoolRC', 'INTERPHONE'],
  'Marca': ['QSPORTPEAK', 'BOSEMAN', 'GoolRC', 'Interphone'],
  'Material del interior': ['ABS, Poliestireno expandido',
   'Poliestireno expandido',
   'Algodón',
   'Algodón']}]